In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_test = tf.keras.utils.to_categorical(y_test, 10)
y_train = tf.keras.utils.to_categorical(y_train, 10)

In [0]:

model=Sequential()
model.add(Conv2D(64, (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(256, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(784,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", 
        metrics=["accuracy"])
model.summary()

epochs = 12
batch_size = 80
datagen = ImageDataGenerator(rotation_range=10,  zoom_range = 0.1, 
                             width_shift_range=0.1,  height_shift_range=0.1)  



train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size=batch_size)

history = model.fit_generator(train_gen, 
                              epochs = epochs, 
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = x_test.shape[0] // batch_size)

# model.fit(x_train, y_train, epochs=12, batch_size=200, validation_data=(x_test, y_test), verbose=1)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 128)        

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model.evaluate(x_test, y_test)


10000/10000 [==============================] - 1s 87us/sample - loss: 0.0139 - acc: 0.9953


[0.013856665517588499, 0.9953]

In [0]:
model.save('cnn_model-e.h5')


In [0]:
new_model = tf.keras.models.load_model('cnn_model.h5')

In [0]:
import pandas as pd
test_data = pd.read_csv('test.csv')
test_data = test_data.values
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

test_data = test_data.astype('float32')

test_data /= 255


In [0]:
pred = new_model.predict(test_data)
print(pred, pred.shape)

[[2.93163596e-17 1.70464456e-21 1.00000000e+00 ... 6.72783182e-17
  6.98461601e-12 3.20942087e-18]
 [9.99994278e-01 5.04864671e-14 6.95545907e-07 ... 1.18114185e-10
  7.63862857e-08 4.11178320e-07]
 [8.70452954e-09 1.44128156e-12 3.66300228e-05 ... 5.80073731e-08
  1.48253001e-08 9.99962926e-01]
 ...
 [6.03751879e-22 1.64941637e-19 1.41484264e-17 ... 4.22164809e-19
  9.47030269e-16 7.41107886e-14]
 [6.71984992e-08 8.65653826e-10 4.98339034e-07 ... 5.04291165e-07
  1.17461045e-08 9.99990940e-01]
 [1.12933845e-22 1.72299047e-27 1.00000000e+00 ... 9.12541875e-22
  3.74823802e-14 4.01800534e-22]] (28000, 10)


In [0]:
p = np.argmax(pred, axis=1)
final = pd.DataFrame({'ImageId': np.arange(1, 28001), 'Label': p})
final = final.set_index('ImageId')
final.to_csv('submission.csv')